# Predicting House Sale Prices
This project delves into linear regression and how to improve it by data cleaning and using cross validation in order to reduce RMSE. We work with a dataset containing housing data from Ames, Iowa, United States from 2006 to 2010 and seek to predict the Sale Price of houses.  
We'll initially import some libraries into our environment and predefine functions we'll later build upon. We'll start with just a simple linear regression model on the dataset:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

data = pd.read_csv('datasets/AmesHousing.tsv', delimiter='\t')

def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460].select_dtypes(include=['integer', 'float'])
    test = df[1460:].select_dtypes(include=['integer', 'float'])
    features = train.columns.drop('SalePrice')
    
    lr = LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predict = lr.predict(test[features])
    mse = mean_squared_error(predict, test['SalePrice'])
    rmse = np.sqrt(mse)
    
    return rmse

transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

# Feature Engineering
We'll proceed to start cleaning up the data by:
1. Removing columns containing more than 5% missing values
2. Dropping columns that are text based that contain any missing values
3. Fill missing values in numerical columns with the most common value of the entire column  
  
Note that these changes can be reverted if we deem missing columns usefull/necessary to carry out a satisfactory regression model.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

In [3]:
# Deleting any column containing more than 5% missing entries

missing = data.isnull().sum()
to_drop_missing = missing[(missing > len(data)/20)].sort_values()
data = data.drop(to_drop_missing.index, axis=1)

In [4]:
# Dropping text columns with 1 or more missing entries

text_missing_counts = data.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

data = data.drop(text_missing_counts[(text_missing_counts > 0)].index, axis=1)

In [5]:
# Filling missing numerical column values with the most common value of it's respective column

missing = data.select_dtypes(include=['int', 'float']).isnull().sum()
missing = missing[(missing > 0)].sort_values()
vals_to_replace = data[missing.index].mode().to_dict(orient='records')[0]
vals_to_replace

{'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Bsmt Unf SF': 0.0,
 'Total Bsmt SF': 0.0,
 'Garage Cars': 2.0,
 'Garage Area': 0.0,
 'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Mas Vnr Area': 0.0}

In [6]:
data = data.fillna(vals_to_replace)
data.isnull().sum().value_counts()

0    64
dtype: int64

Knowing the Year Built and Year Sold we can make a new feature describing the amount of time it took from being built to being sold, which will make our analysis easier:

In [7]:
years_sold = data['Yr Sold'] - data['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

Similarly we can do the same with houses remodelled and the time it took to sell afterwards:

In [8]:
years_remod = data['Yr Sold'] - data['Year Remod/Add']
years_remod[years_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [9]:
# Add new features to the dataset and delete rows with negative values
data['Years Before Sale'] = years_sold
data['Years After Remod'] = years_remod

data = data.drop([1702,2180,2181], axis=0)

data = data.drop(['Yr Sold','Year Remod/Add'], axis=1)

We'll now drop columns that we feel are not useful for machine learning, or leak data about the final sale of the house:

In [10]:
# Not useful for ML
data = data.drop(['PID', 'Order'], axis=1)

## Leak info of final sale
data = data.drop(['Mo Sold', 'Sale Condition', 'Sale Type'], axis=1)

Let's update transform_features() with this code we've developed and see if the RMSE has decreased.

In [11]:
def transform_features(data):
    missing = data.isnull().sum()
    to_drop_missing = missing[(missing > len(data)/20)].sort_values()
    data = data.drop(to_drop_missing.index, axis=1)
    
    text_missing_counts = data.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    data = data.drop(text_missing_counts[(text_missing_counts > 0)].index, axis=1)
    
    missing = data.select_dtypes(include=['int', 'float']).isnull().sum()
    missing = missing[(missing > 0)].sort_values()
    vals_to_replace = data[missing.index].mode().to_dict(orient='records')[0]
    data = data.fillna(vals_to_replace)
    
    years_sold = data['Yr Sold'] - data['Year Built']
    years_remod = data['Yr Sold'] - data['Year Remod/Add']
    data['Years Before Sale'] = years_sold
    data['Years After Remod'] = years_remod
    
    data = data.drop([1702,2180,2181], axis=0)
    data = data.drop(['Mo Sold', 'Sale Condition', 'Sale Type','PID', 'Order','Yr Sold','Year Remod/Add','Yr Sold'], axis=1)
    
    return data

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460].select_dtypes(include=['integer', 'float'])
    test = df[1460:].select_dtypes(include=['integer', 'float'])
    features = train.columns.drop('SalePrice')
    
    lr = LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predict = lr.predict(test[features])
    mse = mean_squared_error(predict, test['SalePrice'])
    rmse = np.sqrt(mse)
    
    return rmse

data = pd.read_csv("datasets/AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.367312413066

# Feature Selection

In [12]:
%matplotlib inline
import seaborn as sns
num_data = transform_df.select_dtypes(include=['int', 'float'])

corr_coeffs = num_data.corr()['SalePrice'].abs().sort_values()
corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years After Remod    0.534985
Full Bath            0.546118
Year Built           0.558490
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice 

In [13]:
corr_coeffs = corr_coeffs[corr_coeffs > 0.35]
corr_coeffs

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years After Remod    0.534985
Full Bath            0.546118
Year Built           0.558490
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [14]:
## Drop columns with less than 0.35 correlation with SalePrice
transform_df = transform_df.drop(corr_coeffs[corr_coeffs < 0.35].index, axis=1)

We define all nominal features that should be categorical. For each column depending on its' type:
* If numerical we'll encode them to be categorical, as numbers do not have any semantic meaning to them
* If there are too many numerical values we'll remove it from the dataset, as adding dummy columns back will be labourious

In [15]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

# Features that are still in our dataframe 

nom_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        nom_cols.append(col)
        
nom_cols

['MS SubClass',
 'MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [19]:
# Number of unique values in these columns
unique_counts = transform_df[nom_cols].apply(lambda x: len(x.value_counts())).sort_values()

# Cutoff any columns that have more than 10 unique values
drop_cols = unique_counts[unique_counts > 10].index
transform_df = transform_df.drop(drop_cols, axis=1)

In [20]:
# Converting text columns to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')

# Dummy columns, then add back to the dataframe
transform_df = pd.concat([
    transform_df, pd.get_dummies(transform_df.select_dtypes(include=['category']))]
    , axis=1).drop(text_cols,axis=1)

We'll again update our functions and also add a k parameter that controls any cross validation that occurs making use of the KFold function

In [33]:
from sklearn.model_selection import KFold

def transform_features(data):
    missing = data.isnull().sum()
    to_drop_missing = missing[(missing > len(data)/20)].sort_values()
    data = data.drop(to_drop_missing.index, axis=1)
    
    text_missing_counts = data.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    data = data.drop(text_missing_counts[(text_missing_counts > 0)].index, axis=1)
    
    missing = data.select_dtypes(include=['int', 'float']).isnull().sum()
    missing = missing[(missing > 0)].sort_values()
    vals_to_replace = data[missing.index].mode().to_dict(orient='records')[0]
    data = data.fillna(vals_to_replace)
    
    years_sold = data['Yr Sold'] - data['Year Built']
    years_remod = data['Yr Sold'] - data['Year Remod/Add']
    data['Years Before Sale'] = years_sold
    data['Years After Remod'] = years_remod
    
    data = data.drop([1702,2180,2181], axis=0)
    data = data.drop(['Mo Sold', 'Sale Condition', 'Sale Type','PID', 'Order','Yr Sold','Year Remod/Add','Yr Sold'], axis=1)
    
    return data

def select_features(df):
    num_data = df.select_dtypes(include=['int', 'float'])
    corr_coeffs = num_data.corr()['SalePrice'].abs().sort_values()
    df = df.drop(corr_coeffs[corr_coeffs < 0.35].index, axis=1)

    nom_cols = []
    for col in nominal_features:
        if col in df.columns:
            nom_cols.append(col)
        
    unique_counts = df[nom_cols].apply(lambda x: len(x.value_counts())).sort_values()
    drop_cols = unique_counts[unique_counts > 10].index
    df = df.drop(drop_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')

    df = pd.concat([
    df, pd.get_dummies(df.select_dtypes(include=['category']))]
    , axis=1).drop(text_cols,axis=1)
    
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df, k):
    lr = LinearRegression()
    features = df.columns.drop('SalePrice')
    num_df = df.select_dtypes(include=['integer', 'float'])
    
    if k ==0:
        train = df[:1460]
        test = df[1460:]
        lr.fit(train[features],train['SalePrice'])
        predict = lr.predict(test[features])
        mse = mean_squared_error(predict, test['SalePrice'])
        rmse = np.sqrt(mse)
    
        return rmse

    
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
            
        return avg_rmse
        

data = pd.read_csv("datasets/AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[26566.04026918346, 36903.06301303401, 25246.865533336502, 25307.93446822479]


28505.975820944688

# Conclusion
We've explored several methods to achieve more accurate predictions of the Sale Price of houses and was able to halve the RMSE by using KFold cross-validation using 4 folds.  
Further optimization could consist of researching on feature selection